In [1]:
import sys
import os
sys.path.append('../') # root directory

In [ ]:
!pip install openpyxl

In [2]:
from constants import constant
import pandas as pd
import numpy as np
import constants.constant as constant
import utils.commonutils as utils
import utils.scraperutils as scraperutils
from constants.namedtuples import Column
from write.JsonFileWriter import write
from read.JsonFileReader import parse
from pathlib import Path

In [3]:
SP500_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
NASDAQ100_URL = 'https://en.wikipedia.org/wiki/NASDAQ-100'
OUT_DIR = Path('../output')

In [4]:
def extract_wiki_table(table_soup):
    '''
    quirks -> all 'th'.stripped_strings[0] are columns, tr.stripped_strings gives row data
    '''
    th_list = table_soup.find_all('th')
    columns = [list(th.stripped_strings)[0] for th in th_list] #stripped[0] because of \n in some
    print(f'Colums:\n{columns}')
    tbody = table_soup.find('tbody')
    rows = tbody.find_all('tr')
    data = [dict(zip(columns, list(row.stripped_strings))) for row in rows[1:]]
    return data

# SP500

In [5]:
soup = scraperutils.get_beautiful_soup(SP500_URL)
table_soup = soup.find("table", id="constituents")
sp500 = extract_wiki_table(table_soup)
sp500_file = f'SP500_Wiki_{utils.current_datetime_str()}'
write(OUT_DIR/sp500_file, sp500)
pd.DataFrame(parse(OUT_DIR/sp500_file)).head()

Colums:
['Symbol', 'Security', 'SEC filings', 'GICS', 'GICS Sub Industry', 'Headquarters Location', 'Date first added', 'CIK', 'Founded']


,Symbol,Security,SEC filings,GICS,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


# NASDAQ100

In [6]:
soup = scraperutils.get_beautiful_soup(NASDAQ100_URL)
table_soup = soup.find("table", id="constituents")
nasdaq100 = extract_wiki_table(table_soup)
nasdaq100_file = f'NASDAQ100_Wiki_{utils.current_datetime_str()}'
write(OUT_DIR/nasdaq100_file, nasdaq100)
pd.DataFrame(parse(OUT_DIR/nasdaq100_file)).head(10)

Colums:
['Company', 'Ticker']


,Company,Ticker
0,Activision Blizzard,ATVI
1,Adobe Inc.,ADBE
2,Advanced Micro Devices,AMD
3,Alexion Pharmaceuticals,ALXN
4,Align Technology,ALGN
5,Alphabet Inc.,(Class A)
6,Alphabet Inc.,(Class C)
7,Amazon.com,AMZN
8,Amgen,AMGN
9,Analog Devices,ADI


In [10]:
pd.DataFrame(nasdaq100).to_csv(OUT_DIR/f'NASDAQ100.csv')
pd.DataFrame(sp500).to_csv(OUT_DIR/f'SP500.csv')